<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>

<h1 align=center><font size = 5>System for clustering similar neighborhoods of NYC and Toronto city</font></h1>


## Introduction
Background:

A multinational company specializing in IT services is based in NYC (New York City). The company is known best in its industry to attract the most diverse employees from the top universities in the country; a majority of the employees are foreign individuals that commence employment on an H1-B visa. The H1-B visa is a lottery-based program and hence, oftentimes, the company’s employees end up not being picked in the lottery and having to move out of the US. In an effort to increase its employee retention which in many ways contributes to the growth of the organization, the company has asked its Global Mobility department to come up with a process to reassign its employees that did not receive an H1-B visa to another country where the company’s subsidiary is located. Global Mobility understood from experience that moving is an arduous process. Global Mobility would like to make this process more efficient and less challenging for the employees. This project is focusing on one of the many steps involved in an employee relocation process by developing a system, that helps to find a neighborhood for their new home in a new city that aligns best with their current lifestyle. To achieve this goal Clustering concept of Machine Learning is used. The developed model will result in clustering similar neighborhoods. This model will then be used to predict the cluster of neighborhoods from Toronto to see which neighborhoods of Toronto and NYC are similar in terms of attributes used from Foursquare database.

Problem:

While the company offers its transferring employees access to real estate brokers to help them find accommodation in the new city, the employees are not always familiar with the neighborhoods of the city and as such, don't know which preferences to indicate to their brokers. To do so, Global Mobility planned to build a system with the location data of all the cities outside the US where the company’s subsidiaries are located. The transferring employees will be asked to indicate their current or preferred location/neighborhoods in a questionnaire. Based on these responses, Global Mobility will then leverage its system to match the employee to a location in the new city that best fits their profile and criteria of preferred neighborhoods. This information will bring more certainty in transferring employees as well as reduce the efforts of the real estate company.

Interest: 

Major multinational companies that have Global Mobility, or a similar department would be interested in building or buying such a system. Such a system can also help employees that are not only moving out because of their visa issues but also moving for temporary work assignments or employees going for secondments.



## Importing libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.12.5          |   py36h5fab9bb_1         143 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         240 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.1.0-pyhd3deb0d_0

The following packages will be UPDATED:

  certifi                          2020.12.5-py36h5fab9bb_0 --> 2020.12.5-py36h5fab9bb_1



geopy-2.1.0          | 64 KB     

<a id='item1'></a>


## 1. Download and Explore Dataset


Neighborhood has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood. 


In [2]:
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
print('Data downloaded!')

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
    
neighborhoods_data = newyork_data['features']

# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

#looping through the data and filling the dataframe one row at a time.
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

#Examining the resulting dataframe.
print (neighborhoods['Borough'].unique())
borough_list= neighborhoods['Borough'].unique()

print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)


Data downloaded!
['Bronx' 'Manhattan' 'Brooklyn' 'Queens' 'Staten Island']
The dataframe has 5 boroughs and 306 neighborhoods.


#### Using geopy library to get the latitude and longitude values of New York City.


In [3]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


#### Creating a map of New York with neighborhoods superimposed on top.


In [11]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

#### Defining Foursquare Credentials and Version


In [4]:
CLIENT_ID = '1XEGUBCCNKIUTDR5IFADER3LCESU0SFXSAH3NUY1W0UYAKUT'
#'ZOAKLSXN2R2BERIJWISFG1BMHW3QYOQF1IASNUBP4E4UGSMC' # your Foursquare ID
CLIENT_SECRET = 'OZKVOC3V1CCRYZLDQBJBBOLWAAO2G4HIXRJ4EYWU4YMVM5UQ'
#'SMYJDGRDAHGJVUDCOX4FSC40MBTG03JWLE2IB1URL5FYSMIF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1XEGUBCCNKIUTDR5IFADER3LCESU0SFXSAH3NUY1W0UYAKUT
CLIENT_SECRET:OZKVOC3V1CCRYZLDQBJBBOLWAAO2G4HIXRJ4EYWU4YMVM5UQ


<a id='item2'></a>


## 2. Exploratory Data Analysis (Exploring Neighborhoods of NYC)

In [5]:
# Function to repeat the same process to all the neighborhoods in New York

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        #print (url)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

# Creating a new dataframe called ny_venues
ny_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

#### checking how many venues were returned for each neighborhood

In [6]:
ny_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allerton,28,28,28,28,28,28
Annadale,12,12,12,12,12,12
Arden Heights,5,5,5,5,5,5
Arlington,5,5,5,5,5,5
Arrochar,19,19,19,19,19,19
Arverne,20,20,20,20,20,20
Astoria,98,98,98,98,98,98
Astoria Heights,12,12,12,12,12,12
Auburndale,18,18,18,18,18,18


#### Unique categories can be curated from all the returned venues


In [7]:
print('There are {} uniques categories.'.format(len(ny_venues['Venue Category'].unique())))

There are 430 uniques categories.


<a id='item3'></a>


### Transforming data and Analyzing Neighborhood


In [8]:
# one hot encoding
ny_onehot = pd.get_dummies(ny_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ny_onehot['Neighborhood'] = ny_venues['Neighborhood']

# move neighborhood column to the first column
fixed_columns = [ny_onehot.columns[-1]] + list(ny_onehot.columns[:-1])
ny_onehot = ny_onehot[fixed_columns]

#ny_onehot.head()
#ny_onehot.to_csv('ny_onehot.csv')
print (ny_onehot.shape)


(10047, 430)


#### Grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [9]:
ny_grouped = ny_onehot.groupby('Neighborhood').mean().reset_index()
#ny_grouped.to_csv('ny_grouped.csv')
#ny_grouped.columns
#ny_grouped.shape
ny_grouped.head()

,Neighborhood,Yoga Studio,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Border Crossing,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Basketball Court,College Bookstore,College Cafeteria,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Medical Supply Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Nightlife Spot,Non-Profit,Noodle House,North Indian Re

Function to sort the venues in descending order to have a visual insight.


In [10]:
def return_most_common_venues(row, num_top_venues):
    #print (row)
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = ny_grouped['Neighborhood']

for ind in np.arange(ny_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ny_grouped.iloc[ind, :], num_top_venues)

#Displaying the top 10 venues for each neighborhood.
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,Pizza Place,Deli / Bodega,Supermarket,Intersection,Smoke Shop,Gas Station,Fried Chicken Joint,Spa,Food,Bus Station
1,Annadale,Pizza Place,Cosmetics Shop,American Restaurant,Bakery,Train Station,Restaurant,Sushi Restaurant,Diner,Dance Studio,Food
2,Arden Heights,Pool,Pharmacy,Playground,Pizza Place,Coffee Shop,Women's Store,Fast Food Restaurant,Entertainment Service,Ethiopian Restaurant,Event Space
3,Arlington,Bus Stop,Intersection,Boat or Ferry,Deli / Bodega,Grocery Store,Filipino Restaurant,Event Space,Exhibit,Eye Doctor,Factory
4,Arrochar,Italian Restaurant,Pizza Place,Deli / Bodega,Bus Stop,Athletics & Sports,Hotel,Mediterranean Restaurant,Middle Eastern Restaurant,Food Truck,Outdoors & Recreation


In [11]:
ny_grouped_clustering = ny_grouped.drop(['Neighborhood'], 1)
ny_grouped_clustering.head()

,Yoga Studio,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Border Crossing,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Basketball Court,College Bookstore,College Cafeteria,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Medical Supply Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Nightlife Spot,Non-Profit,Noodle House,North Indian Restaurant,Offi

<a id='item4'></a>


## 3. Training the model on NYC data


#### Trying _k_-means algorithm to form clusters

Finding best k in _k_-means clustering method 

In [ ]:
from scipy.spatial.distance import cdist

distortions = [] 
inertias = [] 
mapping1 = {} 
mapping2 = {} 
K = range(1,25) 
  
for k in K: 
    #Building and fitting the model 
    kmeanModel = KMeans(n_clusters=k).fit(ny_grouped_clustering) 
    kmeanModel.fit(ny_grouped_clustering)     
      
    distortions.append(sum(np.min(cdist(ny_grouped_clustering, kmeanModel.cluster_centers_, 
                      'euclidean'),axis=1)) / ny_grouped_clustering.shape[0]) 
    inertias.append(kmeanModel.inertia_) 
  
    mapping1[k] = sum(np.min(cdist(ny_grouped_clustering, kmeanModel.cluster_centers_, 
                 'euclidean'),axis=1)) / ny_grouped_clustering.shape[0] 
    mapping2[k] = kmeanModel.inertia_ 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(10,8))
plt.plot(K, inertias, 'bx-') 
plt.xlabel('Values of K') 
plt.ylabel('Distortion') 
plt.title('The Elbow Method using Distortion')
plt.show() 

Training the _k_-means model for best value of _k_

In [12]:
# set number of clusters
kclusters = 4

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ny_grouped_clustering)

# check cluster labels generated for each row in the dataframe
#kmeans.labels_

#kmeans
#ny_grouped_clustering.shape

#### Trying DBSCAN algorithm to form clusters

In [62]:
from sklearn.cluster import DBSCAN 

epsilon = 0.3
minimumSamples = 15
db = DBSCAN(eps=epsilon, min_samples=minimumSamples).fit(ny_grouped_clustering)
labels = db.labels_
labels

array([ 0,  0,  0,  0,  0, -1,  0,  0, -1,  0,  0,  0, -1,  0,  0, -1,  0,
       -1,  0,  0, -1,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,
        0,  0,  0,  0, -1, -1,  0,  0, -1,  0,  0,  0,  0, -1,  0,  0, -1,
        0])

Creating a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.


In [13]:
# adding clustering labels
try:
    neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_.astype(int))
except:
    neighborhoods_venues_sorted['Cluster Labels'] = kmeans.labels_.astype(int)

In [14]:
ny_merged = neighborhoods

# merge ny_grouped with latitude/longitude for each neighborhood
ny_merged = ny_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

#ny_merged.head()
#ny_merged.to_csv('ny_merged.csv')

### Result of data after training the data


In [15]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ny_merged['Latitude'], ny_merged['Longitude'], ny_merged['Neighborhood'], ny_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>


# 4. Predicting data of Toronto city on _k_-means trained model

### Cleaning and Preprocessing testing data

In [16]:
toronto_onehot = pd.read_csv('toronto_onehot.csv')
del toronto_onehot['Unnamed: 0']
#toronto_onehot.head()

toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()

#toronto_grouped.head()

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

### Aligning columns of new data with trained model requirements

In [17]:
cols_ny =ny_grouped_clustering.columns
#print ('tmp_col_ny: ', tmp_col_ny)
cols_tor = toronto_grouped_clustering.columns
#print ('tmp_col_tor: ', tmp_col_tor)
drop_cols = list (set(cols_tor) - set(cols_ny))
#print (len(drop_cols))
include_cols = list (set(cols_ny) - set(cols_tor))
#print (len(include_cols))

print (ny_grouped_clustering.shape)
print (toronto_grouped_clustering.shape)

#df = pd.merge(df,df2[['Key_Column','Target_Column']],on='Key_Column', how='left')
df_toronto = toronto_grouped_clustering
#df_toronto.to_csv ('tmp.csv')
#df.drop(['column_nameA', 'column_nameB'], axis=1, inplace=True)
df_toronto.drop(drop_cols, axis=1, inplace=True)
#df_toronto.to_csv ('tmp.csv')

r,c = df_toronto.shape
#print (r)
L = np.zeros([r,len(include_cols)], dtype = float)
#print (L)

(302, 429)
(39, 231)


In [18]:
df_predict = pd.concat([df_toronto, pd.DataFrame(L, columns=include_cols)], axis=1)

print(df_predict.shape)
df_predict.dropna(how='all', inplace = True)
df_predict.to_csv ('df_predict.csv')

(39, 429)


### Predicting Toronto city data

In [19]:
yhat = kmeans.predict(df_predict)
#print (yhat)

In [25]:
toronto_venues_sorted = pd.read_csv ('neighbourhoods_venues_sorted.csv')
toronto_venues_sorted.shape
del toronto_venues_sorted ['Unnamed: 0']

toronto_venues_sorted.shape
toronto_venues_sorted.insert(0, 'Cluster Labels', yhat)

toronto_merged = pd.read_csv('df_can_tor.csv')

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(toronto_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

#toronto_merged.head() # check the last columns!
#print (toronto_merged.shape)

## 5. Results

Neighborhoods of NYC with top 10 venues

In [42]:
df_ny_ny =ny_merged.loc[ny_merged['Cluster Labels'] == 0, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]
df_ny_ny['City'] = 'NYC'

# move neighborhood column to the first column
fixed_columns = [df_ny_ny.columns[-1]] + list(df_ny_ny.columns[:-1])
df_ny_ny = df_ny_ny[fixed_columns]

# Applied custom sorting on data to get better insights
df_ny_ny = pd.read_csv ('df_ny_ny.csv')

df_ny_ny.head(25)
#df_ny_ny.to_csv('df_ny_ny.csv')

,City,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,NYC,Central Harlem,African Restaurant,Seafood Restaurant,Gym / Fitness Center,Chinese Restaurant,Cosmetics Shop,American Restaurant,Bar,French Restaurant,Spa,Gym
1,NYC,Lower East Side,Art Gallery,Chinese Restaurant,Café,Japanese Restaurant,Latin American Restaurant,Bakery,Coffee Shop,Park,Juice Bar,Mediterranean Restaurant
2,NYC,Red Hook,Art Gallery,Seafood Restaurant,Park,Bar,Brewery,Farm,American Restaurant,Bagel Shop,Café,Pizza Place
3,NYC,Briarwood,Arts & Crafts Store,Deli / Bodega,Bus Station,Playground,Plaza,Coffee Shop,Indian Restaurant,Convenience Store,Gym,Exhibit
4,NYC,Paerdegat Basin,Auto Garage,Food,Bus Line,Gym,Business Service,Asian Restaurant,Women's Store,Field,Event Space,Exhibit
5,NYC,Gramercy,Bagel Shop,Bar,Italian Restaurant,Pizza Place,American Restaurant,Playground,Mexican Restaurant,Cocktail Bar,Grocery Store,Sandwich Place
6,NYC,Williamsburg,Bagel Shop,Bar,Pizza Place,Coffee Shop,Italian Restaurant,Gym,Steakhouse,Sandwich Place,Pet Store,Park
7,NYC,Madison,Bagel Shop,Deli / Bodega,Hobby Shop,Pilates Studio,Pizza Place,Dessert Shop,Restaurant,Italian Restaurant,Daycare,Field
8,NYC,Randall Manor,Bagel Shop,Park,Field,Entertainment Service,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant
9,NYC,Little Italy,Bakery,Café,Chinese Restaurant,Cocktail Bar,Italian Restaurant,Mediterranean Restaurant,Sandwich Place,Salon / Barbershop,Hotel,Bubble Tea Shop


Neighborhoods of Toronto with top 10 venues

In [43]:
#toronto_merged.head(20)
df_tr_tr = toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[3] + list(range(7, toronto_merged.shape[1]))]]
df_tr_tr['City'] = 'Toronto'

# move neighborhood column to the first column
fixed_columns = [df_tr_tr.columns[-1]] + list(df_tr_tr.columns[:-1])
df_tr_tr = df_tr_tr[fixed_columns]

# Applied custom sorting on data to get better insights
df_tr_tr = pd.read_csv ('df_tr_tr.csv')

df_tr_tr.head(25)
#df_tr_tr.to_csv('df_tr_tr.csv')

,City,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Terminal,Boutique,Bar,Coffee Shop,Boat or Ferry,Sculpture Garden,Rental Car Location,Harbor / Marina,Plane
1,Toronto,The Beaches,Asian Restaurant,Pub,Trail,Health Food Store,Neighborhood,Dog Run,Diner,Discount Store,Distribution Center,Donut Shop
2,Toronto,"Dufferin, Dovercourt Village",Bakery,Pharmacy,Bar,Brewery,Bank,Liquor Store,Grocery Store,Park,Music Venue,Pet Store
3,Toronto,"Little Portugal, Trinity",Bar,Coffee Shop,Men's Store,Café,Vegetarian / Vegan Restaurant,Asian Restaurant,Restaurant,Beer Store,Italian Restaurant,Japanese Restaurant
4,Toronto,"Parkdale, Roncesvalles",Breakfast Spot,Gift Shop,Dessert Shop,Cuban Restaurant,Eastern European Restaurant,Dog Run,Bar,Italian Restaurant,Restaurant,Movie Theater
5,Toronto,"Business reply mail Processing Centre, South C...",Brewery,Skate Park,Park,Gym / Fitness Center,Comic Shop,Restaurant,Light Rail Station,Pizza Place,Garden Center,Recording Studio
6,Toronto,"University of Toronto, Harbord",Café,Bar,Japanese Restaurant,Bookstore,Bakery,Yoga Studio,Pub,Beer Bar,Beer Store,Sandwich Place
7,Toronto,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Yoga Studio,Grocery Store,Pet Store,Performing Arts Venue,Nightclub,Italian Restaurant,Intersection
8,Toronto,"Runnymede, Swansea",Café,Coffee Shop,Pub,Pizza Place,Sushi Restaurant,Italian Restaurant,Yoga Studio,French Restaurant,Latin American Restaurant,Indie Movie Theater
9,Toronto,"Kensington Market, Chinatown, Grange Park",Café,Coffee Shop,Vietnamese Restaurant,Mexican Restaurant,Vegetarian / Vegan Restaurant,Dessert Shop,Grocery Store,Bar,Caribbean Restaurant,Gaming Cafe


## <h3 align="center"> © IBM Corporation 2020. All rights reserved. <h3/>
